### Sentiment Model Training Script

In [ ]:
# Import necessary libraries
# Training model on GPU
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
import accelerate

# Check GPU status
print("Using GPU: ", torch.cuda.is_available())  # Should print True
print("GPU Name: ", torch.cuda.get_device_name(0))  # Should show NVIDIA GPU

# Load GoEmotions dataset
dataset = load_dataset("go_emotions")

# Ensure labels are integers, not lists
def preprocess_labels(example):
    if isinstance(example["labels"], list) and len(example["labels"]) > 0:
        label = example["labels"][0]  # Get the first label
        label = min(label, 26)  # Ensure label is in range 0-26
        example["labels"] = int(label)  # Convert to integer
    else:
        example["labels"] = 0  # Default label if missing
    return example

# Apply label preprocessing
dataset = dataset.map(preprocess_labels)

train_labels = [ex["labels"] for ex in dataset["train"]]
val_labels = [ex["labels"] for ex in dataset["validation"]]

print("Unique train labels (after fix):", set(train_labels))
print("Unique validation labels (after fix):", set(val_labels))

# Load the tokenizer for RoBERTa-Base
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define number of labels (27 emotions from GoEmotions)
num_labels = 27

# Load RoBERTa-base with classification head
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)

# Training arguments (Optimized for GPU training)
training_args = TrainingArguments(
    output_dir="./my_sentiment_analysis_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,  # Increase batch size for GPU
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Enables Mixed Precision for faster training on GPU
)

%env CUDA_LAUNCH_BLOCKING=1
# Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save trained model
trainer.save_model("my_sentiment_analysis_model")
tokenizer.save_pretrained("my_sentiment_analysis_model")


### Sample Test

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the trained three-class model from your local directory
model_name = 'jaynwanze/interview_sentiment_model'  # update if necessary
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define a threshold for uncertainty (e.g., if the score difference between the top two is less than 0.1)
THRESHOLD = 0.1

# Define a simplified mapping for a three-class model:
id2label = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

# Create the sentiment analysis pipeline
sentiment_analyzer = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Sample texts to test
sample_texts = [
    # Positive feedback examples
    "The candidate demonstrated excellent learning agility by quickly mastering necessary skills in graphic design and marketing under a tight deadline. This adaptability directly contributed to the team's success and exceeded typical expectations.",
    "The candidate showed strong flexibility by stepping into the marketing director's role on short notice. Adapting to new responsibilities and learning new skills on the fly indicates a high level of flexibility.",
    "The candidate likely maintained composure given the successful outcome under what would have been a high-pressure situation, indicating good emotional resilience. More specific examples of handling stress would provide clearer insights.",
    # # Neutral/balanced feedback examples
    "The candidate admits to not being the best at learning quickly but shows capability to handle situations that require new knowledge, albeit with external help.",
    "The candidate demonstrated flexibility by adapting to a new challenge and working collaboratively to solve it, though it wasn't an independent effort.",
    "While the candidate does not explicitly describe their emotional state during the situation, their ability to seek help and successfully solve the problem suggests a level of resilience.",
    # Negative feedback examples
    "The candidate admits to a significant struggle with learning new skills and knowledge, relying heavily on others rather than developing personal competence.",
    "Based on the candidate's reliance on others for learning and task completion, it suggests a low level of flexibility in adapting to new roles or responsibilities independently.",
    "There is insufficient information to accurately assess emotional resilience directly; however, the dependency on others might indicate potential challenges in handling stress or pressure independently."
]


# Run sentiment analysis
for text in sample_texts:
    result = sentiment_analyzer(text)
    # Sort results by score in descending order
    sorted_results = sorted(result[0], key=lambda x: x['score'], reverse=True)
    top_result = sorted_results[0]
    second_result = sorted_results[1]



    # Check the score difference between top two predictions
    if top_result['score'] - second_result['score'] < THRESHOLD:
        # If difference is small, flag as 'uncertain' or assign neutral
        predicted_label = "neutral"
        final_score = (top_result['score'] + second_result['score']) / 2  # or any custom logic
    else:
        label_str = top_result['label']
        # Robustly extract the label (if it is numeric or a string)
        if label_str.startswith("LABEL_"):
            predicted_label_id = int(label_str.split('_')[1])
            predicted_label = id2label[predicted_label_id]
        elif label_str.isdigit():
            predicted_label_id = int(label_str)
            predicted_label = id2label[predicted_label_id]
        else:
            # If the label is already final (e.g., "positive")
            predicted_label = label_str
        final_score = top_result['score']

    print(f"Text: {text}")
    print(f"Predicted sentiment: {predicted_label} (score: {final_score:.3f})\n")


c:\Dev\interviewgrade_sentiment_model\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Dev\interviewgrade_sentiment_model\.venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\I586463\.cache\huggingface\hub\models--jaynwanze--interview_sentiment_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administ

Text: The candidate demonstrated excellent learning agility by quickly mastering necessary skills in graphic design and marketing under a tight deadline. This adaptability directly contributed to the team's success and exceeded typical expectations.
Predicted sentiment: positive (score: 0.998)

Text: The candidate showed strong flexibility by stepping into the marketing director's role on short notice. Adapting to new responsibilities and learning new skills on the fly indicates a high level of flexibility.
Predicted sentiment: positive (score: 0.998)

Text: The candidate likely maintained composure given the successful outcome under what would have been a high-pressure situation, indicating good emotional resilience. More specific examples of handling stress would provide clearer insights.
Predicted sentiment: neutral (score: 0.812)

Text: The candidate admits to not being the best at learning quickly but shows capability to handle situations that require new knowledge, albeit with ex

### Fine Tuning

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Check GPU availability
print("Using GPU:", torch.cuda.is_available())

# Load and Prepare the CSV Dataset
csv_file = "interview_fine_tuning_dataset.csv"  # Update with your CSV file path
data_df = pd.read_csv(csv_file)

# Convert the pandas DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(data_df)

# Define label mapping for 3-class sentiment analysis
label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2,
}
id2label = {v: k for k, v in label2id.items()}

# Preprocess Labels: Map string labels to integers
def preprocess_labels(example):
    if example["label"] in label2id:
        example["label"] = label2id[example["label"]]
    else:
        example["label"] = -1  # You can filter these out later if needed
    return example

dataset = dataset.map(preprocess_labels)
dataset = dataset.filter(lambda example: example["label"] != -1)

print("Unique labels in dataset:", set(dataset["label"]))  # Expect {0, 1, 2}

# Load Tokenizer and Tokenize Data
model_checkpoint = "./my_sentiment_analysis_model"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load Pre-trained Model for 3-Class Classification
num_labels = 3  # negative, neutral, positive
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)


# Set Training Arguments
training_args = TrainingArguments(
    output_dir="./domain_finetuned_model2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,              
    learning_rate=5e-6,             
    weight_decay=0.01,
    logging_steps=10,
    fp16=torch.cuda.is_available(),
)

# Create Trainer and Fine-Tune
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # If you have a separate eval split, use that
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

# Save the Fine-Tuned Model and Tokenizer
trainer.save_model("./domain_finetuned_model")
tokenizer.save_pretrained("./domain_finetuned_model")

print("Fine-tuning complete. Model saved to './domain_finetuned_model'.")


Using GPU: False


Filter: 100%|██████████| 809/809 [00:00<00:00, 146504.55 examples/s]


Unique labels in dataset: {0, 1, 2}


Map: 100%|██████████| 809/809 [00:00<00:00, 21809.11 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./my_sentiment_analysis_model and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([27]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([27, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Dev\sentiment_model\.venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\I586463\AppData\Local\Temp\ipykernel_12792\3325663027.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0

{'loss': 1.2752, 'grad_norm': 11.561749458312988, 'learning_rate': 4.929971988795519e-06, 'epoch': 0.1}


...

{'loss': 0.0026, 'grad_norm': 0.044532354921102524, 'learning_rate': 9.803921568627452e-08, 'epoch': 6.86}


                                               
  0%|          | 1/714 [37:14<28:32,  2.40s/it]  

{'loss': 0.0026, 'grad_norm': 0.0409274585545063, 'learning_rate': 2.8011204481792718e-08, 'epoch': 6.96}







































































































                                               
                                              

  0%|          | 1/714 [38:44<28:32,  2.40s/it]  

                                               
100%|██████████| 714/714 [38:37<00:00,  3.25s/it]


{'eval_loss': 0.0015791155165061355, 'eval_runtime': 80.2535, 'eval_samples_per_second': 10.081, 'eval_steps_per_second': 1.271, 'epoch': 7.0}
{'train_runtime': 2317.4296, 'train_samples_per_second': 2.444, 'train_steps_per_second': 0.308, 'train_loss': 0.11362725891647678, 'epoch': 7.0}
Fine-tuning complete. Model saved to './domain_finetuned_model'.
